In [1]:
%matplotlib inline
import quimb as qu
import quimb.tensor as qtn
import tensorflow as tf
import numpy as np
import joblib
import matplotlib.pyplot as plt
import glob
import multiprocessing
import time


In [5]:
tf.__version__, qu.__version__, np.__version__

('2.0.0', '1.3.0', '1.18.5')

$ham = scale_x * \sum_{i=1}^{N-1}{\boldsymbol{\sigma}_{x}^{i}\cdot\boldsymbol{\sigma}_{x}^{i + 1}} + scale_y * \sum_{i=1}^{N-1}{\boldsymbol{\sigma}_{y}^{i}\cdot\boldsymbol{\sigma}_{y}^{i + 1}} + scale_z * \sum_{i=1}^{N-1}{\boldsymbol{\sigma}_{z}^{i}\cdot\boldsymbol{\sigma}_{z}^{i + 1}}$


In [30]:
# set the scales that defines the hamiltonian
scale_x, scale_y, scale_z = 1, 0.5, 0.8
# whether to apply symmetry to the circuit to reduce parameters number
symmetry_apply = False
# number of cpu that are going to use
num_cpus = 1
# the system size 
N = 10
# how many layers are going to be implement (1, 2, ... layers)
layers = 5
# how many runs for each set up
runs = 100

# Define gates


In [3]:
def ry_(param):
    return tf.convert_to_tensor([[tf.cos(param / 2), -tf.sin(param / 2)],
                                 [tf.sin(param / 2),
                                  tf.cos(param / 2)]])


def rz_(param):
    return tf.convert_to_tensor([[tf.exp(-1j * param / 2), 0],
                                 [0, tf.exp(1j * param / 2)]])


def phase_gate_(param):
    return tf.convert_to_tensor([[1, 0], [0, tf.exp(1j * param)]])


def cnot_(reverse=False):
    if reverse:
        return tf.constant(qu.permute(qu.CNOT(), [2, 2], perm=[1, 0]))
    else:
        return tf.constant(qu.CNOT())

def noisy_cnot_(noise, reverse=False):
    noisy_cz = np.sqrt(1j) * qu.expm(1j * qu.kron(
        qu.pauli('Z'), qu.pauli('I')) @ np.kron(qu.pauli('I'), qu.pauli('Z')) *
                                     (np.pi / 4 + noise)) @ qu.kron(
                                         qu.Rz(np.pi / 2), qu.Rz(np.pi / 2))
    if reverse == False:
        noisy_cnot = qu.kron(qu.pauli('I'), qu.hadamard()) @ noisy_cz @ qu.kron(
            qu.pauli('I'), qu.hadamard())
    else:
        noisy_cnot = qu.kron(qu.hadamard(), qu.pauli('I')) @ noisy_cz @ qu.kron(
            qu.hadamard(), qu.pauli('I'))

# The circuit of 
# $$\mathcal{N}(\theta_x, \theta_y, \theta_z) = e^{i\left(\theta_x \sigma_{x} \otimes \sigma_{x} + \theta_y \sigma_{y} \otimes \sigma_{y} + \theta_z \sigma_{z} \otimes \sigma_{z}\right)}$$
the circuit figure can be check on our paper arXiv:2006.09415, or the original paper https://doi.org/10.1103/PhysRevA.69.032315

In [4]:
def N_block(psi, params_block, wires):
    theta_x = scale_x * params_block
    theta_y = scale_y * params_block
    theta_z = scale_z * params_block
    
    theta_x = 2 * theta_x - np.pi / 2
    theta_y = 2 * theta_y - np.pi / 2
    theta_z = 2 * theta_z - np.pi / 2

    psi.gate_(rz_(-np.pi / 2), wires[1], tags='RZ')

    psi.gate_(cnot_(reverse=True), wires, tags='CNOT')

    psi.gate_(rz_(-theta_z), wires[0], tags='RZ')

    psi.gate_(ry_(theta_x), wires[1], tags='RY')

    psi.gate_(cnot_(reverse=False), wires, tags='CNOT')

    psi.gate_(ry_(-theta_y), wires[1], tags='RY')

    psi.gate_(cnot_(reverse=True), wires, tags='CNOT')

    psi.gate_(rz_(np.pi / 2), wires[0], tags='RZ')

# The ansatz circuit
The circuit figure can be check on our paper arXiv:2006.09415

In [5]:
def ansatz(psi, params, num_layer, N):
    # split all the params to the N function params and rotation params
    params_block = params[0]
    params_rot = params[1]
    # copy |psi> manually to avoid any operation happens on the psi0
    psi = psi.copy()

    # apply the gates layer-wise 
    for i in range(num_layer):
        for j in range(0, N, 2):
            N_block(psi,
                       params_block[i][j],
                       wires=[j, j + 1])
        for j in range(1, N - 1, 2):
            N_block(psi,
                   params_block[i][j],
                   wires=[j, j + 1])
        # if apply symmetry rotation params will cut down to half
        # where \theta_{0} = -\theta_{N - 1}
        if symmetry_apply:
            for j in range(N // 2):
                psi.gate_(phase_gate_(params_rot[i][j]), j, tags='ROT')
                psi.gate_(phase_gate_(-params_rot[i][j]), N - j - 1, tags='ROT')
        else:
            for j in range(N):
                psi.gate_(phase_gate_(params_rot[i][j]), j, tags='ROT')
    return psi

# Tensor network fidelity function

In [6]:
def fidelity_tn(state1, state2):
    return np.abs((state1 & state2).contract(all,
                                             optimize='random-greedy',
                                             backend='tensorflow').numpy())**2

# Tensor network expectation value function

In [7]:
def expt_val(psi, H):
    energy = qtn.TensorNetwork(qtn.align_TN_1D(psi.H, H, psi))
    return energy.contract(all, optimize='random-greedy', backend='tensorflow')

# Tool function (to change the inner data of tensors to complex128)

In [8]:
def apply_to_data(*objs):
    for obj in objs:
        for t in obj:
            t.modify(data=tf.constant(t.data, dtype=tf.complex128))

# For ignore the warning information of tensorflow (only use this when you sure the code is ok)

In [9]:
import logging
tf.get_logger().setLevel(logging.ERROR)

# Main training logic 

In [10]:
def train_process(psi0, H, gs, N, num_layer, run, epoch=-1, tol=1e-4):
    print(
        'Run={}, N={}, num_layer={} scale={}/{}/{} symmetry={} training begin.'.
        format(run, N, num_layer, scale_x, scale_y, scale_z, symmetry_apply))
    # to avoid the random params are the same if you using multi-process
    # mannully reset the numpy random seed
    np.random.seed()

    # to store all the intermediate results
    params_gd = []
    expt_gd = []
    fidelity_gd = []

    # randomly generate the initial params
    params = tf.Variable(np.random.randn(2, num_layer, N))

    # define the optimizer
    # here we use adam with amsgrad applied
    opt = tf.keras.optimizers.Adam(learning_rate=0.01,
                                   beta_1=0.9,
                                   beta_2=0.999,
                                   epsilon=1e-8,
                                   amsgrad=True)

    for index in range(1, epoch + 1):
        with tf.GradientTape() as tape:
            # cast params into complex data type otherwise tensorflow won't work
            newparams = tf.complex(params, tf.constant(0, dtype=tf.float64))
            # evolve the system
            psi = ansatz(psi0, newparams, num_layer, N)
            # calculate the loss, in this case, the expectation value
            loss = expt_val(psi, H)
        # tensorflow gradient functions
        gradients = tape.gradient(loss, params)
        opt.apply_gradients(zip([gradients], [params]))

        # record the intermediate results
        params_gd.append(params.numpy())
        expt_gd.append(4 * loss.numpy().real)
        fidelity_gd.append(fidelity_tn(gs, psi))

        if index % 50 == 0:
            print(
                "Run={}, N={}, num_layer={}, scale={}/{}/{}, step {}: expt: {:.3f}, fidelity: {:.3f}"
                .format(run, N, num_layer, scale_x, scale_y, scale_z, index,
                        expt_gd[-1], fidelity_gd[-1]))

    # Store the data
    joblib.dump(
        params_gd,
        'xxz/random_scheme_data/params_run={}_N={}_layer={}_epoch={}_scale={}{}{}_symmetry={}'
        .format(run, N, num_layer, (-1 if epoch == int(1e8) else epoch),
                scale_x, scale_y, scale_z, symmetry_apply))
    joblib.dump(
        expt_gd,
        'xxz/random_scheme_data/expt_run={}_N={}_layer={}_epoch={}_scale={}{}{}_symmetry={}'
        .format(run, N, num_layer, (-1 if epoch == int(1e8) else epoch),
                scale_x, scale_y, scale_z, symmetry_apply))
    joblib.dump(
        fidelity_gd,
        'xxz/random_scheme_data/fidelity_run={}_N={}_layer={}_epoch={}_scale={}{}{}_symmetry={}'
        .format(run, N, num_layer, (-1 if epoch == int(1e8) else epoch),
                scale_x, scale_y, scale_z, symmetry_apply))
    return fidelity_gd[-1], index

In [31]:
# multiprocessing pool. 
pool = multiprocessing.Pool(num_cpus)

# not knowing why, but this part need to be out of the for-loop 
# to make the multi-process work. Your case might be varied.
# define the |psi0> which is |0101...01>
psi0 = qtn.MPS_computational_state('01' * (N // 2))
# define the ham which scale settings
H = qtn.MPO_ham_heis(N, j=(scale_x, scale_y, scale_z))
# calculate the ground state then convert it to tensor network
gs = qtn.Dense1D(
        qu.groundstate(qu.ham_heis(N, j=(scale_x, scale_y, scale_z), cyclic=False, sparse=True)))
# convert the inner data type to tf.complex128 to make the code work
apply_to_data(psi0, H, gs)

In [14]:
# run defines how many runs the code is going to run
for run in range(1, runs + 1):
    for num_layer in range(1, layers + 1):
        if symmetry_apply:
            params_number = num_layer * ((N * 3) // 2 - 1)
        else:
            params_number = num_layer * (2 * N - 1)
        # we set the training iteration to be 50 * number_of_params
        epoch = params_number * 50
        try:
            pool.apply_async(train_process, args=(psi0, H, gs, N, num_layer, run, epoch, 1e-4, ))
        except:
            print("Error!!!")
# have to run this 2 lines after using multi-processing pool
pool.close()
pool.join()